Notebook where the scripts and functions to generate the stroke mask for each video are shown.

In [17]:
import numpy as np
import cv2 as cv
import cv2

def fingerprint(path, filename, dirr):
    '''
    Function to generate the trace mask of a video with optical flow

    path:     str: video address
    filename: str: name of the image to save
    dirr:     str: directory where you save the image
    
    '''
    cap = cv.VideoCapture(path)
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                           qualityLevel = 0.3,
                           minDistance = 7,
                           blockSize = 7 )

    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15, 15),
                      maxLevel = 2,
                      criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    # Create color
    color = np.array([0,255,0])

    # Take first frame and find corners in it
    for j in range(4):
        ret, old_frame = cap.read()
        old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    #-----------
    height, width, _ = old_frame.shape
    ny = height/10
    nx = width/15
    
    # Point selecction
    p0 = np.array([[[ny,nx]]])

    t=0
    for j in range(15):
        t+=1
        n=0
        y = ny*t
        for i in range(10):
            n+=1
            x = nx*n
            p0 = np.append(p0,[np.array([[y,x]])],axis= 0)

    p0 = np.float32(p0)

    # Black background
    imbk = cv2.imread('black.jpg') 
    imbk = cv2.resize(imbk, (width,height))

    count=0
    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    while True:
        ret, frame = cap.read()
        if not ret:
            #print('No frames grabbed!')
            break
        count+=1
        
    # Count frame n° 4
        if count > 4:
            imbk2 = imbk.copy()
            frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
      
        # Calculate optical flow
            p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
      
        # Select good points
            if p1 is not None:
                good_new = p1[st==1]
                good_old = p0[st==1]

        # draw the tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()

                try:
                    mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color.tolist(), 1, lineType = 50) 
                    frame = cv.circle(frame, (int(a), int(b)), 5, color.tolist(), -1) 
                    imbk2 = cv.circle(imbk2, (int(a), int(b)), 5, color.tolist(), -1) 
                except:
                    pass

            img1 = cv.add(imbk2, mask)

        # Now update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

    cv2.imwrite(dirr + filename + '.jpg', img1)


In [18]:
import pandas as pd
train = pd.read_csv("finish_data_train_final.csv")
train

,dir,movement_label,movement_value
0,./../data/trailer_zoom/tt0444850_shot_0014.mp4,Push,2
1,./../data/trailer_zoom/tt0444850_shot_0015.mp4,Push,2
2,./../data/trailer_zoom/tt0444850_shot_0016.mp4,Push,2
3,./../data/trailer_zoom/tt2005173_shot_0002.mp4,Push,2
4,./../data/trailer_zoom/tt2005173_shot_0014.mp4,Push,2
...,...,...,...
17051,./../data/pull_push2/tt1156398/shot_0004.mp4,Pull,3
17052,./../data/pull_push2/tt3292146/shot_0003.mp4,Pull,3
17053,./../data/pull_push2/tt5157030/shot_0004.mp4,Pull,3
17054,./../data/pull_push2/tt3087700/shot_0028.mp4,Pull,3


In [19]:
import pandas as pd
test = pd.read_csv("finish_data_test_final.csv")
test

,dir,movement_label,movement_value
0,./../data/val/tt3097956/shot_0024.mp4,Static,4
1,./../data/val/tt3603824/shot_0001.mp4,Static,4
2,./../data/val/tt4047132/shot_0075.mp4,Static,4
3,./../data/val/tt4325152/shot_0071.mp4,Static,4
4,./../data/val/tt3351004/shot_0020.mp4,Static,4
...,...,...,...
571,./../data/test/tt2231554/shot_0059.mp4,Push,1
572,./../data/test/tt3309038/shot_0007.mp4,Push,1
573,./../data/test/tt3229778/shot_0060.mp4,Push,1
574,./../data/test/tt4930760/shot_0045.mp4,Push,1


In [20]:
from tqdm import tqdm
entrada = open('data_test_trace.csv', 'wt')
for j in tqdm(range(len(test)), desc='Loop'):
    try:
        path_video = str(test['dir'][j])
        movement = str(test['movement_label'][j])
        mv = str(test['movement_value'][j])
        entrada.write(f'{j}.jpg' + "," + movement + ',' + mv + "\n")
        fingerprint(path_video, f'{j}', './datatest/') 
    except:
        print('Error', path_video, j)
        pass
entrada.close()

Loop: 100%|██████████| 576/576 [06:32<00:00,  1.47it/s]
